# AML CLI v2 と YAML による学習

[00_LoadData.ipynb](./00_LoadData.ipynb) を実行して、あらかじめデータセットの登録をおこなってください。

> Note : データセットのアップロード・登録を CLI v2 と YAML で記述できます。

## Azure へのログイン

In [ ]:
!az login

In [ ]:
!az account set -s {AZURE_SUBSCRIPTION_ID}

## AML ワークスペースの環境設定

In [1]:
my_resource_group = "{AML-RESOURCE-GROUP-NAME}"
my_workspace = "{AML-WORSPACE-NAME}"

## [Optional] AML コンピュート (クラスター) の作成

demo-cpucluster1 の名前の AML compute cluster がない場合には、下記を実行して、この名前の AML compute cluster を作成します。<br>
(既に存在する場合には実行不要です。)

In [2]:
!az ml compute create --name demo-cpucluster1 \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace \
  --type amlcompute \
  --min-instances 0 \
  --max-instances 1 \
  --size Standard_D2_v2

Command group 'ml compute' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "id": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST2020426-ML/providers/Microsoft.MachineLearningServices/workspaces/ws01/computes/demo-cpucluster1",
  "idle_time_before_scale_down": 120,
  "location": "eastus",
  "max_instances": 1,
  "min_instances": 0,
  "name": "demo-cpucluster1",
  "network_settings": {},
  "provisioning_state": "Succeeded",
  "resourceGroup": "TEST2020426-ML",
  "size": "STANDARD_D2_V2",
  "ssh_public_access_enabled": true,
  "tier": "dedicated",
  "type": "amlcompute"
}


## [Optional] Environment (環境) の作成

AML 上に arima-env の Environment (環境) を作成していない場合には下記を実行します。<br>
(既に作成済の場合は不要です。)

In [3]:
%%writefile arima-env-config.yml
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: arima-env
image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
conda_file: scripts/arima-env.yml
description: This is example

Writing arima-env-config.yml


In [4]:
!az ml environment create --file arima-env-config.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml environment' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "conda_file": {
    "channels": [
      "anaconda",
      "conda-forge"
    ],
    "dependencies": [
      "python=3.6.9",
      {
        "pip": [
          "azureml-defaults",
          "statsmodels==0.12.2",
          "scipy",
          "scikit-learn",
          "joblib",
          "numpy",
          "pandas",
          "matplotlib",
          "seaborn",
          "numpy"
        ]
      }
    ],
    "name": "arima-env"
  },
  "creation_context": {
    "created_at": "2022-04-26T05:14:43.758937+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User",
    "last_modified_at": "2022-04-26T05:14:43.758937+00:00",
    "last_modified_by": "Tsuyoshi Matsuzaki",
    "last_modified_by_type": "User"
  },
  "description": "This is example",
  "id": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST2020426-ML/provid

## 学習実行

In [8]:
%%writefile train-job-config.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
code: scripts
command: >-
  python train.py
  --data_file ${{inputs.transaction_ts2013}}
inputs:
  transaction_ts2013:
    type: uri_file
    path: azureml:transaction_ts2013:1
environment: azureml:arima-env@latest
compute: azureml:demo-cpucluster1
display_name: arima-mlops-remote-cli
experiment_name: arima-mlops-remote-cli
description: Train demo

Writing train-job-config.yml


In [9]:
!az ml job create --file train-job-config.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml job' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "code": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST2020426-ML/providers/Microsoft.MachineLearningServices/workspaces/ws01/codes/831618de-8420-44d7-b2de-925151530c8c/versions/1",
  "command": "python train.py --data_file ${{inputs.transaction_ts2013}}",
  "compute": "azureml:demo-cpucluster1",
  "creation_context": {
    "created_at": "2022-04-26T05:18:11.543628+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User"
  },
  "description": "Train demo",
  "display_name": "arima-mlops-remote-cli",
  "environment": "azureml:arima-env:1",
  "environment_variables": {},
  "experiment_name": "arima-mlops-remote-cli",
  "id": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST2020426-ML/providers/Microsoft.MachineLearningServices/workspaces/ws01/jobs/mighty_brake_4g86h53rsg",
  "inputs": {
    

## Model のダウンロード

下記コマンドで、モデルをローカルにダウンロードし、作成されたモデルの確認・評価などをおこなえます。<br>
(```job_name``` には、上記で出力されている ```name``` を設定します。)

In [10]:
job_name = "mighty_brake_4g86h53rsg"

In [11]:
!az ml job download --name $job_name \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml job' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
